In [1]:
import numpy as np
from random import random

from pytket.circuit import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket.circuit import QControlBox
from pytket.circuit import CircBox

In [50]:
# m is precision of answer
def iter_circ(m_bits, eigenstate: Circuit, unitary: Circuit) -> Circuit:
    # create base of circuit to return
    iter_circ: Circuit = Circuit(1, m_bits + 1)
    qreg = iter_circ.get_q_register("q")
    creg = iter_circ.get_c_register("c")
    ancil = qreg[0]
    n_state_prep_qubits = eigenstate.n_qubits
    sreg = iter_circ.add_q_register("psi", n_state_prep_qubits)
    iter_circ.add_circuit(eigenstate, list(sreg))

    # Create a controlled unitary with a single control qubit
    unitary.name = "U"
    controlled_u_gate = QControlBox(CircBox(unitary), 1)

    # sub-circuit to compute creg[j] (the j'th bit of the answer)
    for j in range(m_bits, 0, -1):
        # add hadamard to ancillary qubit
        iter_circ.H(0)
        # add unitaries
        for _ in range(2 ** (j - 1)):
            iter_circ.add_qcontrolbox(controlled_u_gate, [ancil] + list(sreg))

        # add rotations
        for expn in range(m_bits + 1 - j, 1, -1):
            phase = - 2 * np.pi / (2 ** expn)
            # this is the problematic line, I believe
            iter_circ.U1(phase, 0, condition=creg[expn + j - 1])
            print(expn, expn + j - 1)
        iter_circ.H(0)
        iter_circ.Measure(ancil, creg[j])
        iter_circ.Reset(0)

    # dummy just in case
    iter_circ.Measure(ancil, creg[0])
    return iter_circ

## TESTING

In [86]:
#input_angle = 0.5 
# From 0 to 1

# copy testcase from tutorial
input_angle = 0.70
initial_state = Circuit(1).X(0) # Make sure to just add quantum channels here
unitary = Circuit(1).U1(input_angle, 0)
circuit_iter_to_render = iter_circ(4, eigenstate=initial_state, unitary=unitary)

render_circuit_jupyter(circuit_iter_to_render)

2 4
3 4
2 3
4 4
3 3
2 2


In [4]:
from pytket.extensions.nexus import NexusBackend, QuantinuumConfig, Nexus
from datetime import datetime
from pytket.extensions.quantinuum import QuantinuumBackend, QuantinuumAPIOffline
api_offline = QuantinuumAPIOffline()

In [5]:
phase_est_project = Nexus().new_project(f"IPE_test10 - {datetime.now()}")

configuration = QuantinuumConfig(device_name="H1-1LE", user_group="iQuHACK_2024")
remote_backend = NexusBackend(
    backend_config= configuration, 
    project= phase_est_project,
)
local_backend = QuantinuumBackend(device_name="H1-1LE", api_handler = api_offline)


New project created: IPE_test10 - 2024-02-04 13:20:09.342387

Started using project with name: IPE_test10 - 2024-02-04 13:20:09.342387


In [89]:
comp_circ = local_backend.get_compiled_circuit(circuit_iter_to_render)

In [91]:
n_shots = 1000
result = remote_backend.process_circuit(comp_circ, n_shots)

In [92]:
result

ResultHandle('68d09cf1-92f7-477b-9fb1-6f04801bb86a', 1927910)

In [93]:
result = remote_backend.get_result(result)

In [85]:
result.get_counts()

Counter({(0, 0, 0, 0, 0): 1,
         (0, 0, 0, 0, 1): 1,
         (0, 0, 0, 1, 0): 894,
         (0, 0, 1, 1, 0): 14,
         (0, 0, 1, 1, 1): 41,
         (0, 1, 0, 0, 0): 6,
         (0, 1, 0, 1, 0): 7,
         (0, 1, 1, 0, 0): 12,
         (0, 1, 1, 0, 1): 19,
         (0, 1, 1, 1, 0): 5})

In [73]:
result.get_counts().most_common()[0][0]

(0, 0, 0, 0, 1, 1, 1, 1, 1)

In [74]:
from pytket.backends.backendresult import BackendResult


In [75]:
def single_phase_from_backendresult(result: BackendResult) -> float:
    # Extract most common measurement outcome
    basis_state = result.get_counts().most_common()[0][0]
    # ignore the c[0] line
    basis_state = basis_state[1:]
    bitstring = "".join([str(bit) for bit in basis_state])
    integer_j = int(bitstring, 2)

    # Calculate theta estimate
    return integer_j / (2 ** len(bitstring))

In [76]:
theta = single_phase_from_backendresult(result)

In [77]:
theta
# i am getting 0.12109 here....consistently

0.12109375

In [78]:
error = round(abs(input_angle - (2 * theta)), 3)
print(error)

0.488
